In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch
#from "Tools.py" import 
seed=42

### load dataset

In [2]:
banco = pd.read_csv("Life_Expectancy_Data.csv")
x = banco[["infant deaths","Year"]]
#x = banco["infant deaths"]
y =banco["Life expectancy "]
x = x[~y.isna()]
y = y[~y.isna()]

### create folders

In [3]:
cv = KFold(n_splits=10,shuffle =True,random_state=seed)
cross = cv.split(x)
x_train = []
x_test = []
y_train = []
y_test = []
for train_i,test_i in cross:
    x_train.append(x.iloc[train_i,:])
    x_test.append(x.iloc[test_i,:])
    y_train.append(y.iloc[train_i])
    y_test.append(y.iloc[test_i])

,infant deaths,Year
0,62,2015
1,64,2014
2,66,2013
3,69,2012
4,71,2011
...,...,...
2932,28,2005
2933,27,2004
2935,25,2002
2936,25,2001


### define model

In [9]:
x = x_train[0]
y = y_train[0]
model = torch.nn.Linear(2,1)
ann = ANN(model)
a = ann.fit(x,y)
a

OrderedDict([('weight', tensor([[0.3637, 0.3470]])), ('bias', tensor([0.5233]))])
0 : 1.5497709512710571
1 : 1.5472819805145264
2 : 1.5447990894317627
3 : 1.5423223972320557
4 : 1.5398517847061157
5 : 1.537387728691101
6 : 1.5349301099777222
7 : 1.5324785709381104
8 : 1.530034065246582
9 : 1.5275959968566895


In [232]:
torch.randn(128, 20).shape

torch.Size([128, 20])

In [241]:
model = torch.nn.Sequential(torch.nn.Linear(2,1))
yhat = model(a.x)
yhat = yhat.view(len(yhat))

In [243]:
yhat.shape


torch.Size([2379])

In [285]:
loss_fn = torch.nn.MSELoss(reduction="mean")
loss = loss_fn(yhat[:506],a.y[:506])
loss

tensor(nan, grad_fn=<MseLossBackward0>)

In [287]:
y[:506]

0      65.0
1      59.9
2      59.9
3      59.5
4      59.2
       ... 
620    53.2
621    52.6
622    52.7
623    52.9
624     NaN
Name: Life expectancy , Length: 506, dtype: float64

In [98]:
a.loss_fn(a.model(a.x).view(len(a.x)),a.y).item()

0.3163611590862274

In [8]:
class ANN:
    def __init__(self,model,seed=0,intercv = 3,n_epoch=10,batch_size=64,lr=0.001, device = "cpu"):
        self.seed = seed
        self.intercv = intercv
        self.model = model
        self.batch_size = batch_size
        self.lr = lr
        self.scale_x = StandardScaler()
        self.scale_y = StandardScaler()
        self.device = device
        self.n_epoch = n_epoch
        self.losses = []
        self.optim = torch.optim.Adam(model.parameters(),lr=self.lr)
        self.loss_fn = torch.nn.MSELoss(reduction="mean")
        self.model.dtype = torch.float

    def fit(self, x, y):
        x_ = self.scale_x.fit_transform(np.array(x))
        y_ = self.scale_y.fit_transform(np.array(y).reshape(-1, 1)).reshape(len(y))
        x_ = x_.copy()
        y_ = y_.copy()
        x_ = torch.as_tensor(x_,dtype=torch.float)
        y_ = torch.as_tensor(y_,dtype=torch.float)
        torch.manual_seed = self.seed
        #train = torch.utils.data.TensorDataset(x_,y_)
        #train = torch.utils.data.DataLoader(dataset=train,batch_size=self.batch_size,shuffle=True)
        self.x = x_
        self.y = y_
        print(self.model.state_dict())
        self._train(x_,y_)
           # self.losses.append(np.mean(mini_batch_loss))
           # if epoch%100==0:
           #     print(str(epoch) + " erro = " + str(self.losses[epoch]))
                
        
        return self
    
    def _train(self,x,y):
        self.model.train()
        for epoch in range(self.n_epoch):
            self.x = x
            self.yhat = self.model(x)
            loss = self.loss_fn(self.yhat.view(len(self.yhat)),y)
            self.losses.append(loss.item())
            loss.backward()
            self.optim.step()
            self.optim.zero_grad()
            print(str(epoch) + " : "+ str(loss.item()))

    def predict(self, x):
        return self.model.predict(x)
    
    def predict_proba(self,x):
        return self.model.predict_proba(x)
    

In [56]:
model = torch.nn.Linear(1,1)
optim = torch.optim.Adam(model.parameters(),lr=self.lr)

NameError: name 'x_' is not defined